## STAT628 Module 3 Yelp Data 
### Brian Tsai 
#### mtsai36@wisc.edu

### Importing Required Packages

In [ ]:
import numpy as np
import pandas as pd
import os
import json
from scipy import stats as st 
from collections import Counter
import matplotlib.pyplot as py
import seaborn as sns 
#from langdetect import detect_langs
import re

### Reading Business json data 

In [4]:
data = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/business.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        data.append(json.loads(line))

# Mexican restaurants 
mex = [] 
for i in range(len(data)): 
    if data[i]['categories'] != None: 
        if 'Mex' in data[i]['categories']:
            mex.append(data[i])

# Mexican restuarants that are open            
mex_open = []
for i in mex: 
    if i['is_open'] == 1: 
        mex_open.append(i)


# Business id for open Mexican restuarants         
mex_id = []
for i in mex_open: 
    mex_id.append(i['business_id'])

# Tacobell restaurants with >= 10 reviews  
tacobell = [] 
tacobell_id = [] 
for i in mex_open: 
    if 'Taco Bell' in i['name'] and i['review_count'] >=10: 
        tacobell.append(i)
        tacobell_id.append(i['business_id'])

    
# Chipotle restaurants with >= 10 reviews 
chipotle = [] 
chipotle_id = [] 
for i in mex_open:
    if 'Chipotle Mex' in i['name'] and i['review_count'] >= 10:
        chipotle.append(i) 
        chipotle_id.append(i['business_id'])


## User json file 

In [ ]:
user = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/user.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        user.append(json.loads(line))
df = pd.DataFrame(user)
print(len(user))
df.head()


## Tips json file 

In [ ]:
tacobell_tips = []
chipotle_tips = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/tip.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        x = json.loads(line)
        if x['business_id'] in tacobell_id: 
            tacobell_tips.append(x)
        if x['business_id'] in chipotle_id: 
            chipotle_tips.append(x)

## Reviews 

In [6]:
# Tacobell and Chipotle Reviews         
tacobell_reviews = []
chipotle_reviews = [] 
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/review.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        x = json.loads(line)
        if x['business_id'] in tacobell_id: 
            tacobell_reviews.append(x)
        if x['business_id'] in chipotle_id: 
            chipotle_reviews.append(x)        

df = pd.DataFrame(tacobell_reviews)
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,k9vlSSUStwY2DcjM8Rinnw,7C8RwnXqNhj6mYxowI53sA,Of6xu3pY3eHe2yhiyz2dvg,4.0,0,1,0,"Literally not a thing, anything at all, in my...",2017-05-13 05:57:03
1,AtNXA1mJ8BJR1OK7sM68Ug,2Y01picIp0O3uvo45l_jMQ,xyLIKm-pXy-zc29weExTZQ,4.0,0,0,0,I've lived in the area for 5 years now. This t...,2015-08-19 16:30:22
2,HTvz23J8kC-B9ZaJ_Y9v0Q,D6dvJfPX3Pbcn5JQJZx2xA,Of6xu3pY3eHe2yhiyz2dvg,1.0,2,1,1,This is probably the worst Taco Bell I've ever...,2016-03-07 02:48:02
3,O2KoZYlcLv2QYR5C1WzQiQ,84H-Yk7MaqhCAGdk7UDjuA,8yLc33AAXB9-xn7OG4Firw,1.0,0,0,0,"Saturday evening, 10:30pm, closed. No explanat...",2018-07-08 02:30:44
4,706rWMW5g-IpRT1bGD85GQ,1ENBskBHIvWPaiF-wcZ8Ew,VG__jPnmg4Dvz7e_55iTcw,2.0,0,0,0,The food has good taste but if u want a bad st...,2007-12-14 17:50:49


## Preliminary Analysis

In [ ]:
# How the stars are distributed in the reviews 
# How the stars are distributed among the restaurants in the chain 


# Overall stars rating statistics
tacobell_stars = []
chipotle_stars = []
for i in tacobell_reviews:
    tacobell_stars.append(i['stars'])
for j in chipotle_reviews:
    chipotle_stars.append(j['stars'])

tacobell_avg_rating = sum(tacobell_stars) / len(tacobell_reviews)
chipotle_avg_rating = sum(chipotle_stars) / len(chipotle_reviews)

tacobell_stars_dist = Counter(tacobell_stars)
chipotle_stars_dist = Counter(chipotle_stars)

# Stars rating distribution for each tacobell restaurant
d1 = {}
for i in tacobell_id:
    d1[i] = Counter([j['stars'] for j in tacobell_reviews if j['business_id'] == i])

# Average stars rating for each tacobell restaurant
def f(i):
    """ Returns avg rating for inputted restaurant id"""
    c = 0
    s = 0
    for r in tacobell_reviews:
        if r['business_id'] == i:
            c += r['stars']
            s += 1
    return c/s 

tacobell_res_avg_rating = {}
for i in tacobell_id:
    tacobell_res_avg_rating[i] = f(i)

# Stars rating distribution for each chipotle restaurant
d2 = {}
for i in chipotle_id:
    d2[i] = Counter([j['stars'] for j in chipotle_reviews if j['business_id'] == i])

# Average stars rating for each tacobell restuarant
def g(i):
    """ Returns avg rating for inputted restaurant id"""
    c = 0
    s = 0
    for r in chipotle_reviews:
        if r['business_id'] == i:
            c += r['stars']
            s += 1
    return c/s 

chipotle_res_avg_rating = {}
for i in chipotle_id:
    chipotle_res_avg_rating[i] = g(i)


## Tacobell Plots

In [ ]:
# Number of tacobell reviews per month plot
path = 'C:/Users/tsai_/Desktop/U Wisconsin - Madison/Courses/2nd year/Fall 2021/STAT 628/Module 3'
df_t = pd.DataFrame(tacobell_reviews)
df_t['date'] = pd.to_datetime(df_t['date'])
df_t = df_t.set_index('date')
py.plot(df_t['text'].resample('M').count())
py.xlabel('Year')
py.ylabel('Number of reviews')
py.title('Number of reviews per month')
py.savefig(path + 'tacobell reviews per month.png')
py.show()
py.close()


# Distribution of Tacobell Customer ratings
ax = sns.barplot(data=df_t, x='stars', y='stars', estimator=lambda x: len(x) / len(df_t) * 100)
ax.set(ylabel='Percent')
py.title('Distribution of Customer Rating')
py.savefig(path + 'Tacobell customer ratings distribution.png')
py.show()
py.close()

# Average Monthly Customer Rating Tacobell
py.plot(df_t['stars'].resample('M').mean())
py.xlabel('Year')
py.ylabel('Rating')
py.title('Average Monthly Customer Rating')
py.ylim(0, 5)
py.savefig(path + 'tacobell average monthly customer rating.png')
py.show()
py.close()



## Chipotle Plots

In [ ]:
# Number of chipotle reviews per month plot
path = 'C:/Users/tsai_/Desktop/U Wisconsin - Madison/Courses/2nd year/Fall 2021/STAT 628/Module 3'
df_c = pd.DataFrame(chipotle_reviews)
df_c['date'] = pd.to_datetime(df_c['date'])
df_c = df_c.set_index('date')
py.plot(df_c['text'].resample('M').count())
py.xlabel('Year')
py.ylabel('Number of reviews')
py.title('Number of reviews per month')
py.savefig(path + 'chipotle reviews per month.png')
py.show()
py.close()


# Distribution of Chipotle Customer ratings
ax = sns.barplot(data=df_c, x='stars', y='stars', estimator=lambda x: len(x) / len(df_c) * 100)
ax.set(ylabel='Percent')
py.title('Distribution of Customer Rating')
py.savefig(path + 'Chipotle customer ratings distribution.png')
py.show()
py.close()

# Average Monthly Customer Rating Chipotle
py.plot(df_c['stars'].resample('M').mean())
py.xlabel('Year')
py.ylabel('Rating')
py.title('Average Monthly Customer Rating')
py.ylim(0, 5)
py.savefig(path + 'chipotle average monthly customer rating.png')
py.show()
py.close()

## Preliminary Taco Bell NLP 

In [ ]:
# Checking for reviews not in English 
df = df_t
language = [detect_langs(i) for i in df.text]
languages = [str(i[0]).split(':')[0] for i in language]
df['language'] = languages

# Remove reviews not in English
df = df[df.language == 'en']


# Stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
my_stop_words = set(stopwords.words('english')+list(ENGLISH_STOP_WORDS))

# Nouns and adjectives from Taco Bell reviews
nouns = []
txt = ' '.join(df['text'])
sentences = nltk.sent_tokenize(txt)
for sentence in sentences:
    for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
        if pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS':
            nouns.append(word)

s = ''
for i in nouns:
    s += (i + ' ')

# Noun Word Cloud
from wordcloud import WordCloud
cloud_no_stopword = WordCloud(background_color='white', stopwords=my_stop_words).generate(s)
py.imshow(cloud_no_stopword, interpolation='bilinear')
py.axis('off')
py.savefig(path + 'Tacobell noun word cloud.png')
py.show()
py.close()
    

adj = []
for sentence in sentences:
    for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
        if pos == 'JJ' or pos == 'JJR' or pos == 'JJS':
            adj.append(word)

s1 = ''
for i in adj:
    s1 += (i + ' ')
            
# Adjective Word Cloud 
cloud_no_stopword = WordCloud(background_color='white', stopwords=my_stop_words).generate(s1)
py.imshow(cloud_no_stopword, interpolation='bilinear')
py.axis('off')
py.savefig(path + 'Tacobell adj word cloud.png')
py.show()
py.close()


# Tokenization and Bag of words
from nltk.tokenize import word_tokenize
from nltk import FreqDist
lower_full_text = full_text.lower()
word_tokens = word_tokenize(lower_full_text)
tokens = [] 
for word in word_tokens:
    if word.isalpha() and word not in my_stop_words:
        tokens.append(word)
token_dist = FreqDist(tokens)
dist = pd.DataFrame(token_dist.most_common(20),columns=['Word', 'Frequency'])            
            
    


## Preliminary Chipotle NLP 

In [ ]:
# Checking for reviews not in English 
df = df_c
language = [detect_langs(i) for i in df.text]
languages = [str(i[0]).split(':')[0] for i in language]
df['language'] = languages

# Remove reviews not in English
df = df[df.language == 'en']


# Stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
my_stop_words = set(stopwords.words('english')+list(ENGLISH_STOP_WORDS))

# Nouns and adjectives from Taco Bell reviews
nouns = []
txt = ' '.join(df['text'])
sentences = nltk.sent_tokenize(txt)
for sentence in sentences:
    for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
        if pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS':
            nouns.append(word)

s = ''
for i in nouns:
    s += (i + ' ')

# Noun Word Cloud
from wordcloud import WordCloud
cloud_no_stopword = WordCloud(background_color='white', stopwords=my_stop_words).generate(s)
py.imshow(cloud_no_stopword, interpolation='bilinear')
py.axis('off')
py.savefig(path + 'Chipotle noun word cloud.png')
py.show()
py.close()
    

adj = []
for sentence in sentences:
    for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
        if pos == 'JJ' or pos == 'JJR' or pos == 'JJS':
            adj.append(word)

s1 = ''
for i in adj:
    s1 += (i + ' ')
            
# Adjective Word Cloud 
cloud_no_stopword = WordCloud(background_color='white', stopwords=my_stop_words).generate(s1)
py.imshow(cloud_no_stopword, interpolation='bilinear')
py.axis('off')
py.savefig(path + 'Chipotle adj word cloud.png')
py.show()
py.close()


# Tokenization and Bag of words
from nltk.tokenize import word_tokenize
from nltk import FreqDist
lower_full_text = full_text.lower()
word_tokens = word_tokenize(lower_full_text)
tokens = [] 
for word in word_tokens:
    if word.isalpha() and word not in my_stop_words:
        tokens.append(word)
token_dist = FreqDist(tokens)
dist = pd.DataFrame(token_dist.most_common(20),columns=['Word', 'Frequency'])

## Sentiment Analysis VADER

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# Generate sentiment scores
sentiment_scores = df['text'].apply(sid.polarity_scores)
sentiment = sentiment_scores.apply(lambda x: x['compound'])
monthly_sentiment = sentiment.resample('M').mean()
py.plot(monthly_sentiment, color='blue')
py.axhline(color='red')
py.xlabel('Year')
py.ylabel('Sentiment')
py.title('Average Sentiment Score over time')
py.show()
py.savefig(path + 'Tacobell sentiment score over time.png')
py.close()